In [1]:
#import data preprocessing & visualizing libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from random import random

In [2]:

def preprocessing():
    global df, df_scaled, df_feat
    #Identify the categorical columns in the dataset
    obj_df = df.select_dtypes(include=['object']).copy()
    
    #Encoding the categorical values ( Integer encoding )
    new_df = df.apply(LabelEncoder().fit_transform)
    
    #Randomization of the dataset
    np.random.seed(1000)
    
    # Seperating the Features and Target Columns 
    df_feat = new_df[new_df.columns[1:-1]] # Feature columns
    df_head = new_df[new_df.columns[len(new_df.columns)-1]]  # Target variable 
    
    # Feature Scalling using StandardScaler funstion in 
    scaler = StandardScaler()
    StandardScaler(copy=True,with_mean=True,with_std=True)
    scaler.fit(df_feat)
    scaled_features = scaler.transform(df_feat)
    df_scaled = pd.DataFrame(scaled_features,columns=df_feat.columns)
    
    # split data into train+validation set and test set

    # it is required to split the dataset twice to make 
    global X_train, X_test, y_train, y_test,X_val,y_val
    X_train, X_test, y_train, y_test = train_test_split(df_scaled, df_head, test_size=0.2, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2
    tk.Label(window, text='Preprocessing Completed').grid(row=2, column=1)
    return X_train, X_test, y_train, y_test,X_val,y_val


In [3]:
class MLP(object):
    """A Multilayer Perceptron class.
    """

    def __init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=1):
        """Constructor for the MLP. Takes the number of inputs,
            a variable number of hidden layers, and number of outputs
        Args:
            num_inputs (int): Number of inputs
            hidden_layers (list): A list of ints for the hidden layers
            num_outputs (int): Number of outputs
        """

        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs

        # create a generic representation of the layers
        layers = [num_inputs] + hidden_layers + [num_outputs]

        # create random connection weights for the layers
        weights = []
        for i in range(len(layers) - 1):
            w = np.random.rand(layers[i], layers[i + 1])
            weights.append(w)
        self.weights = weights

        # save derivatives per layer
        derivatives = []
        for i in range(len(layers) - 1):
            d = np.zeros((layers[i], layers[i + 1]))
            derivatives.append(d)
        self.derivatives = derivatives

        # save activations per layer
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations


    def forward_propagate(self, inputs):
        """Computes forward propagation of the network based on input signals.
        Args:
            inputs (ndarray): Input signals
        Returns:
            activations (ndarray): Output values
        """

        # the input layer activation is just the input itself
        activations = inputs

        # save the activations for backpropogation
        self.activations[0] = activations

        # iterate through the network layers
        for i, w in enumerate(self.weights):
            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)

            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)

            # save the activations for backpropogation
            self.activations[i + 1] = activations

        # return output layer activation
        return activations


    def back_propagate(self, error):
        """Backpropogates an error signal.
        Args:
            error (ndarray): The error to backprop.
        Returns:
            error (ndarray): The final error of the input
        """

        # iterate backwards through the network layers
        for i in reversed(range(len(self.derivatives))):

            # get activation for previous layer
            activations = self.activations[i+1]

            # apply sigmoid derivative function
            delta = error * self._sigmoid_derivative(activations)

            # reshape delta as to have it as a 2d array
            delta_re = delta.reshape(delta.shape[0], -1).T

            # get activations for current layer
            current_activations = self.activations[i]

            # reshape activations as to have them as a 2d column matrix
            current_activations = current_activations.reshape(current_activations.shape[0],-1)

            # save derivative after applying matrix multiplication
            self.derivatives[i] = np.dot(current_activations, delta_re)

            # backpropogate the next error
            error = np.dot(delta, self.weights[i].T)


    def train(self, inputs, targets, epochs, learning_rate,validation_data,validation_freq = 10):
        """Trains model running forward prop and backprop
        Args:
            inputs (ndarray): X
            targets (ndarray): Y
            epochs (int): Num. epochs we want to train the network for
            learning_rate (float): Step to apply to gradient descent
        """
        self.validation_freq = validation_freq
        self.train_acc_history = []
        self.valid_acc_history = []
        global training_acc
        training_acc = []
        # now enter the training loop
        for i in range(epochs):
            true_predictions = 0

            # iterate through all the training data
            for j, input in enumerate(inputs):
                target = targets[j]

                # activate the network!
                output = self.forward_propagate(input)

                error = target - output

                self.back_propagate(error)

                # now perform gradient descent on the derivatives
                # (this will update the weights
                self.gradient_descent(learning_rate)

                # keep track of the MSE for reporting later
                true_predictions += int((target==1 and output>0.5) or (target==0 and output<0.5))
            
            # Epoch complete, report the training error
            training_acc.append(("Accuracy: {} at epoch {}".format(true_predictions / len(inputs), i+1)))
            #print("Accuracy: {} at epoch {}".format(true_predictions / len(inputs), i+1))
            self.train_acc_history.append(true_predictions / len(inputs))
            
            if self.check_overfitting(validation_data,i):
                print("Edge of overfitting reached! Training terminated")
                break
                
        print("Training complete!")
        print("=====")


    def gradient_descent(self, learningRate=1):
        """Learns by descending the gradient
        Args:
            learningRate (float): How fast to learn.
        """
        # update the weights by stepping down the gradient
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights += derivatives * learningRate
            
    def _sigmoid(self, x):
        """Sigmoid activation function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """

        y = 1.0 / (1 + np.exp(-x))
        return y


    def _sigmoid_derivative(self, x):
        """Sigmoid derivative function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """
        return x * (1.0 - x)


    def check_overfitting(self,validation_data,ii):
        inputs,targets = validation_data[0],validation_data[1]
        true_preds = 0
        for i, input in enumerate(inputs):
            target = targets[i]
            # activate the network!
            output = self.forward_propagate(input)
            # keep track of the MSE for reporting later
            true_preds += int((target==1 and output>0.5) or (target==0 and output<0.5))
        self.valid_acc_history.append(true_preds/len(inputs))
        if ii%self.validation_freq == 0:
            if len(self.valid_acc_history)!=1:
                if self.valid_acc_history[-1]<self.valid_acc_history[-1-self.validation_freq]:
                    return True
        return False
    
    def plot_acc(self):
        epoches = len(self.train_acc_history)
        plt.plot(list(range(epoches)),self.train_acc_history,label = 'Training')
        
        plt.plot(list(range(epoches)),self.valid_acc_history, label = 'Validation')
        
        plt.xlabel("epoches")
        plt.ylabel("Accuracy")
        plt.title("Accuracy History")
        plt.legend(loc = 'best')
        #File saved in the location where the notebook is stored
        plt.savefig("accuracy.png")
        

In [ ]:
# I used pillow library so  library so that accuracy plot can be displayed on the GUI. 


# In[8]:


get_ipython().system('pip install pillow')


# Please read the README file supplied with this document before running this example.

# In[17]:


import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
import pandas as pd
import numpy as np
from PIL import ImageTk, Image
# import matplotlib
# matplotlib.use('TkAgg')
# import numpy as np
# from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
# from matplotlib.figure import Figure
#from Tkinter import *

window = tk.Tk()
window.geometry("1024x600")
window.resizable(width=True,height=True)
#Window title 
blank_space = " "
window.title(blank_space*80+'MPL Neural Network Binary Classifier')

global  df_scaled
global imglabel
global training_acc


def import_csv_data():
    global v, df
    csv_file_path = askopenfilename()
    v.set(csv_file_path)
    df = pd.read_csv(csv_file_path)
    #return df

    
##########Label
tk.Label(window, text='File Path').grid(row=0, column=0)
v = tk.StringVar()

##########Text boxes 
entry = tk.Entry(window, textvariable=v,width=90).grid(row=0, column=1)

##########Creating  buttons for the window

#Browse the dataset
b = tk.Button(window, text='Browse Data Set',command=import_csv_data).grid(row=0, column=2)
#Preprocessing
b = tk.Button(window, text='Preprocessing',command=preprocessing).grid(row=3, column=1)
#b.pack(fill='x')


######Displaying the preprocessed dataset
def show():
    tempList = np.array(df_scaled.iloc[:,:])
    cols = (df_scaled.columns)
    cols = list(df_scaled.columns)
    listBox = ttk.Treeview(window, columns=cols, show='headings', selectmode='extended')
    for col in cols:
        listBox.heading(col, text=col)
        listBox.column(col, width=40)
    listBox.grid(row=5, column=0, columnspan=len(cols))
    for i, cols in enumerate(tempList, start=1):
        listBox.insert("", "end", values=(cols))
    

#label = tk.Label(window, text="Preprocessing Output", font=("Arial",30)).grid(row=4, columnspan=3)
tk.Button(window, text="Show Values", width=15, command=show).grid(row=4, column=0)


####Entering the layer data and performing the training of the neural network
####Finally displaying the final test accuracy and the accuracy plot

def layers():
    global layer
    value = layer.get()
    value = str(value).split(',')
    value = list(map(int, value))
    # Convert both features and target variables in array to train a network for the sum operation
    innns = np.array(X_train) # assigning the features set 
    targets = np.array(y_train)  # assigning the target variable 
    
    valid_data = (np.array(X_val),np.array(y_val))
    
    # create a Multilayer Perceptron with one hidden layer
    mlp = MLP(df_feat.shape[1], value, 1)
    # train network
    mlp.train(innns, targets, 20, 0.5,valid_data,5)
    
    # create dummy data
    input = X_train #np.array([0.3, 0.1])
    target = y_train  #np.array([0.4])

    # get a prediction
    output = mlp.forward_propagate(input)
    #display the plot
    fig = mlp.plot_acc()
    #display the test, training and validation accuracy
    accuracy_test_validation(mlp)
    # #Setting it up
    try:
        img = ImageTk.PhotoImage(Image.open("accuracy.png"))
        # #Displaying it
        imglabel = tk.Label(window)
        imglabel.config(image=img)
        imglabel.image=img
        imglabel.place(x=600, y=400)
    except FileNotFoundError:
        tk.Label(window, text="Please check the location of the file and the name of the file").place(x=600, y=400)
        print("Please check the location of the file and the name of the file")
        
        
def accuracy_test_validation(mlp):
    # reporting final accuracies
    out_test = mlp.forward_propagate(X_test)
    true_labels = np.array(y_test)
    true_preds = 0
    for i in range(len(true_labels)):
        true_preds += int((true_labels[i]==1 and out_test[i]>0.5) or (true_labels[i]==0 and out_test[i]<0.5))
    print_value = (f"Final Accuracy on the Test data: {true_preds/len(out_test)}")
    tk.Label(window, text=print_value).place(x=300, y=400)
    # reporting final accuracies
    final_validation_acc = mlp.valid_acc_history[-1]
    final_training_acc = mlp.train_acc_history[-1]
    print_value = (f"Final validation Accuracy: {final_validation_acc}\n Final Training Accuracy: {final_training_acc}")
    tk.Label(window, text=print_value).place(x=300, y=420)
    

tk.Label(window, text='layers').grid(row=20, column=0)
layer = tk.StringVar()
##########Text boxes 
layers_data = tk.Entry(window, textvariable=layer,width=40).grid(row=20, column=1)
b = tk.Button(window, text='Confirm', command=layers).grid(row=20, column=2)


###Display the training accuracy at every epoch
def show_training_accuracy():
    tempList = training_acc
    lb = tk.Listbox(window, width=40)
    for col in tempList:
        lb.insert(tk.END, col)
    lb.place(x=20, y=400)

##Label and the button for display
tk.Label(window, text="Training Output", font=("Arial",10)).grid(row=22, column=0)
tk.Button(window, text="Show Training Accuracy", width=20, command=show_training_accuracy).grid(row=20, column=0)

 
#Close button
tk.Button(window, text='Close',command=window.destroy).place(x=1250, y=600) #grid(row=500, column=500)


window.mainloop()


Unable to create process using 'C:\Users\Savio Fernando\anaconda3\python.exe "C:\Users\Savio Fernando\anaconda3\Scripts\pip-script.py" install pillow'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Savio Fernando\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-9993a9c1ee27>", line 149, in show_training_accuracy
    tempList = training_acc
NameError: name 'training_acc' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Savio Fernando\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-9993a9c1ee27>", line 149, in show_training_accuracy
    tempList = training_acc
NameError: name 'training_acc' is not defined
